In [1]:
# sort and split

import pandas as pd

# Load the CSV file
file_path = '/Users/fabrizioferrari/Desktop/OG.csv'
df = pd.read_csv(file_path)

# Ensure 'Fecha Cierre' is in datetime format
df['Fecha Cierre'] = pd.to_datetime(df['Fecha Cierre'], format='%m/%d/%Y', errors='coerce')

# Ensure 'Fecha Colocacion' is in datetime format
df['Fecha Colocacion'] = pd.to_datetime(df['Fecha Colocacion'], format='%m/%d/%Y', errors='coerce')

# Sort by 'Operacion' and then by 'Fecha Cierre' within each 'Operacion'
df_sorted = df.sort_values(by=['Operacion', 'Fecha Cierre'])

# Split the data into two DataFrames based on 'Fecha Colocacion' months
training_split = df_sorted[df_sorted['Fecha Colocacion'].dt.month <= 9]
test_split = df_sorted[df_sorted['Fecha Colocacion'].dt.month >= 10]

# Save the sorted DataFrame to new CSV files
training_set = '/Users/fabrizioferrari/Desktop/final boss/tlon/Training_Set.csv'
test_set = '/Users/fabrizioferrari/Desktop/final boss/tlon/Test_Set.csv'
training_split.to_csv(training_set, index=False)
test_split.to_csv(test_set, index=False)

print(f"Data has been sorted and saved to {training_set} and {test_set}")

/var/folders/6m/k7tj7j8j0z1_ry92lnh0blg00000gn/T/ipykernel_9060/1751686986.py:7: DtypeWarning: Columns (7,17,56,58,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Data has been sorted and saved to /Users/fabrizioferrari/Desktop/final boss/tlon/Training_Set.csv and /Users/fabrizioferrari/Desktop/final boss/tlon/Test_Set.csv


In [2]:
#test set completo con columnas

import pandas as pd

# Load the CSV file
file_path = '/Users/fabrizioferrari/Desktop/final boss/tlon/Test_Set.csv'
df = pd.read_csv(file_path)

# Ensure 'Fecha Cierre' is in datetime format
df['Fecha Cierre'] = pd.to_datetime(df['Fecha Cierre'], format='%Y-%m-%d', errors='coerce')

# Standardize 'Fecha Venta' column: replace typical non-null representations with NaN
df['Fecha Venta'].replace(['None', 'N/A', '', 'nan'], pd.NaT, inplace=True)

# Convert 'Fecha Venta' to datetime, setting errors='coerce' to handle any remaining non-datetime entries
df['Fecha Venta'] = pd.to_datetime(df['Fecha Venta'], errors='coerce')

# Initialize dictionaries to store 'Mora Final' and 'Atraso' columns for each 'Operacion'
mora_final_dict = {}
atraso_dict = {'Atraso30': {}, 'Atraso60': {}, 'Atraso90': {}, 'Atraso120': {}, 'Atraso150': {}, 'Atraso180': {}}

for operacion, group in df.groupby('Operacion'):
    # Determine if the loan was sold
    sold = group['Fecha Venta'].notna().any()
    estado_operacion_10 = (group['ESTADO_OPERACION'] == 10).any()
    estado_operacion_7 = (group['ESTADO_OPERACION'] == 7).any()
    closest_to_present = group.loc[group['Fecha Cierre'].idxmax()]
    moroso = closest_to_present['Atraso'] > 30

    if not sold and estado_operacion_10:
        mora_final = 0
    elif estado_operacion_7 and not moroso:
        mora_final = 0
    elif estado_operacion_7 and moroso:
        # Calculate the Mora Final
        min_cuotas_pend = group[group['CUOTAS_PEND'] > 0]['CUOTAS_PEND'].min()
        valor_cuota = group['Valor Cuota'].iloc[0]
        mora_final_antes_de_venta = min_cuotas_pend * valor_cuota
        mora_final = mora_final_antes_de_venta * 0.5
    else:
        # Calculate the Mora Final
        min_cuotas_pend = group[group['CUOTAS_PEND'] > 0]['CUOTAS_PEND'].min()
        valor_cuota = group['Valor Cuota'].iloc[0]
        mora_final_antes_de_venta = min_cuotas_pend * valor_cuota
        mora_final = mora_final_antes_de_venta * 0.82

    # Store the 'Mora Final' for each 'Operacion'
    mora_final_dict[operacion] = mora_final

    # Determine 'Atraso' levels for each 'Operacion'
    atraso_dict['Atraso30'][operacion] = int(group['Atraso'].max() > 30)
    atraso_dict['Atraso60'][operacion] = int(group['Atraso'].max() > 60)
    atraso_dict['Atraso90'][operacion] = int(group['Atraso'].max() > 90)
    atraso_dict['Atraso120'][operacion] = int(group['Atraso'].max() > 120)
    atraso_dict['Atraso150'][operacion] = int(group['Atraso'].max() > 150)
    atraso_dict['Atraso180'][operacion] = int(group['Atraso'].max() > 180)

# Add the new columns to the original DataFrame
df['Mora Final'] = df['Operacion'].map(mora_final_dict)
df['Pagare'] = df['Cant. Cuotas'] * df['Valor Cuota']
df['Mora Final %'] = df.apply(lambda row: 0 if row['Mora Final'] == 0 else row['Mora Final'] / row['Pagare'], axis=1)
df['Atraso30'] = df['Operacion'].map(atraso_dict['Atraso30'])
df['Atraso60'] = df['Operacion'].map(atraso_dict['Atraso60'])
df['Atraso90'] = df['Operacion'].map(atraso_dict['Atraso90'])
df['Atraso120'] = df['Operacion'].map(atraso_dict['Atraso120'])
df['Atraso150'] = df['Operacion'].map(atraso_dict['Atraso150'])
df['Atraso180'] = df['Operacion'].map(atraso_dict['Atraso180'])

# Save the modified DataFrame to a new CSV file
output_file_path = '/Users/fabrizioferrari/Desktop/final boss/tlon/Test_Set_Columnas.csv'
df.to_csv(output_file_path, index=False)

print(f"Data with 'Mora Final', 'Pagare', 'Mora Final %', and 'Atraso' columns saved to {output_file_path}")


/var/folders/6m/k7tj7j8j0z1_ry92lnh0blg00000gn/T/ipykernel_9060/3718314681.py:7: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Data with 'Mora Final', 'Pagare', 'Mora Final %', and 'Atraso' columns saved to /Users/fabrizioferrari/Desktop/final boss/tlon/Test_Set_Columnas.csv


In [3]:
#training set condensado con columnas vvvvvvvvvvvv

'''import pandas as pd

# Load the CSV file
file_path = '/Users/fabrizioferrari/Desktop/final boss/tlon/Training_Set.csv'
df = pd.read_csv(file_path)

# Ensure 'Fecha Cierre' is in datetime format
df['Fecha Cierre'] = pd.to_datetime(df['Fecha Cierre'], format='%Y-%m-%d', errors='coerce')

# Standardize 'Fecha Venta' column: replace typical non-null representations with NaN
df['Fecha Venta'].replace(['None', 'N/A', '', 'nan'], pd.NaT, inplace=True)

# Convert 'Fecha Venta' to datetime, setting errors='coerce' to handle any remaining non-datetime entries
df['Fecha Venta'] = pd.to_datetime(df['Fecha Venta'], errors='coerce')

# Initialize dictionaries to store 'Mora Final' and 'Atraso' columns for each 'Operacion'
mora_final_dict = {}
atraso_dict = {'Atraso30': {}, 'Atraso60': {}, 'Atraso90': {}, 'Atraso120': {}, 'Atraso150': {}, 'Atraso180': {}}

for operacion, group in df.groupby('Operacion'):
    # Determine if the loan was sold
    sold = group['Fecha Venta'].notna().any()
    estado_operacion_10 = (group['ESTADO_OPERACION'] == 10).any()
    estado_operacion_7 = (group['ESTADO_OPERACION'] == 7).any()
    closest_to_present = group.loc[group['Fecha Cierre'].idxmax()]
    moroso = closest_to_present['Atraso'] > 30

    if not sold and estado_operacion_10:
        mora_final = 0
    elif estado_operacion_7 and not moroso:
        mora_final = 0
    elif estado_operacion_7 and moroso:
        # Calculate the Mora Final
        min_cuotas_pend = group[group['CUOTAS_PEND'] > 0]['CUOTAS_PEND'].min()
        valor_cuota = group['Valor Cuota'].iloc[0]
        mora_final_antes_de_venta = min_cuotas_pend * valor_cuota
        mora_final = mora_final_antes_de_venta * 0.5
    else:
        # Calculate the Mora Final
        min_cuotas_pend = group[group['CUOTAS_PEND'] > 0]['CUOTAS_PEND'].min()
        valor_cuota = group['Valor Cuota'].iloc[0]
        mora_final_antes_de_venta = min_cuotas_pend * valor_cuota
        mora_final = mora_final_antes_de_venta * 0.82

    # Store the 'Mora Final' for each 'Operacion'
    mora_final_dict[operacion] = mora_final

    # Determine 'Atraso' levels for each 'Operacion'
    atraso_dict['Atraso30'][operacion] = int(group['Atraso'].max() > 30)
    atraso_dict['Atraso60'][operacion] = int(group['Atraso'].max() > 60)
    atraso_dict['Atraso90'][operacion] = int(group['Atraso'].max() > 90)
    atraso_dict['Atraso120'][operacion] = int(group['Atraso'].max() > 120)
    atraso_dict['Atraso150'][operacion] = int(group['Atraso'].max() > 150)
    atraso_dict['Atraso180'][operacion] = int(group['Atraso'].max() > 180)

# Add the new columns to the original DataFrame
df['Mora Final'] = df['Operacion'].map(mora_final_dict)
df['Pagare'] = df['Cant. Cuotas'] * df['Valor Cuota']
df['Mora Final %'] = df.apply(lambda row: 0 if row['Mora Final'] == 0 else row['Mora Final'] / row['Pagare'], axis=1)
df['Atraso30'] = df['Operacion'].map(atraso_dict['Atraso30'])
df['Atraso60'] = df['Operacion'].map(atraso_dict['Atraso60'])
df['Atraso90'] = df['Operacion'].map(atraso_dict['Atraso90'])
df['Atraso120'] = df['Operacion'].map(atraso_dict['Atraso120'])
df['Atraso150'] = df['Operacion'].map(atraso_dict['Atraso150'])
df['Atraso180'] = df['Operacion'].map(atraso_dict['Atraso180'])

# Condense the DataFrame by keeping only the first row of each 'Operacion'
df_condensed = df.drop_duplicates(subset='Operacion', keep='first')

# Save the modified DataFrame to a new CSV file
output_file_path = '/Users/fabrizioferrari/Desktop/final boss/tlon/Training_Set_Columnas_Condensed.csv'
df_condensed.to_csv(output_file_path, index=False)

print(f"Data with 'Mora Final', 'Pagare', 'Mora Final %', 'Atraso30', 'Atraso60', 'Atraso90', 'Atraso120', 'Atraso150', and 'Atraso180' saved to {output_file_path}")'''
#training set con mora y rentabilidad anualizada


import pandas as pd

# Load the CSV file
file_path = '/Users/fabrizioferrari/Desktop/final boss/tlon/Training_Set.csv'
df = pd.read_csv(file_path)

# Ensure 'Fecha Cierre' is in datetime format
df['Fecha Cierre'] = pd.to_datetime(df['Fecha Cierre'], format='%Y-%m-%d', errors='coerce')

# Standardize 'Fecha Venta' column: replace typical non-null representations with NaN
df['Fecha Venta'].replace(['None', 'N/A', '', 'nan'], pd.NaT, inplace=True)

# Convert 'Fecha Venta' to datetime, setting errors='coerce' to handle any remaining non-datetime entries
df['Fecha Venta'] = pd.to_datetime(df['Fecha Venta'], errors='coerce')

# Initialize dictionaries to store 'Mora Final' and 'Atraso' columns for each 'Operacion'
mora_final_dict = {}
atraso_dict = {'Atraso30': {}, 'Atraso60': {}, 'Atraso90': {}, 'Atraso120': {}, 'Atraso150': {}, 'Atraso180': {}}

for operacion, group in df.groupby('Operacion'):
    # Determine if the loan was sold
    sold = group['Fecha Venta'].notna().any()
    estado_operacion_10 = (group['ESTADO_OPERACION'] == 10).any()
    estado_operacion_7 = (group['ESTADO_OPERACION'] == 7).any()
    closest_to_present = group.loc[group['Fecha Cierre'].idxmax()]
    moroso = closest_to_present['Atraso'] > 30

    if not sold and estado_operacion_10:
        mora_final = 0
    elif estado_operacion_7 and not moroso:
        mora_final = 0
    elif estado_operacion_7 and moroso:
        # Calculate the Mora Final
        min_cuotas_pend = group[group['CUOTAS_PEND'] > 0]['CUOTAS_PEND'].min()
        valor_cuota = group['Valor Cuota'].iloc[0]
        mora_final_antes_de_venta = min_cuotas_pend * valor_cuota
        mora_final = mora_final_antes_de_venta * 0.5
    else:
        # Calculate the Mora Final
        min_cuotas_pend = group[group['CUOTAS_PEND'] > 0]['CUOTAS_PEND'].min()
        valor_cuota = group['Valor Cuota'].iloc[0]
        mora_final_antes_de_venta = min_cuotas_pend * valor_cuota
        mora_final = mora_final_antes_de_venta * 0.82

    # Store the 'Mora Final' for each 'Operacion'
    mora_final_dict[operacion] = mora_final

    # Determine 'Atraso' levels for each 'Operacion'
    atraso_dict['Atraso30'][operacion] = int(group['Atraso'].max() > 30)
    atraso_dict['Atraso60'][operacion] = int(group['Atraso'].max() > 60)
    atraso_dict['Atraso90'][operacion] = int(group['Atraso'].max() > 90)
    atraso_dict['Atraso120'][operacion] = int(group['Atraso'].max() > 120)
    atraso_dict['Atraso150'][operacion] = int(group['Atraso'].max() > 150)
    atraso_dict['Atraso180'][operacion] = int(group['Atraso'].max() > 180)

# Add the new columns to the original DataFrame
df['Mora Final'] = df['Operacion'].map(mora_final_dict)
df['Pagare'] = df['Cant. Cuotas'] * df['Valor Cuota']
df['Mora Final %'] = df.apply(lambda row: 0 if row['Mora Final'] == 0 else row['Mora Final'] / row['Pagare'], axis=1)
df['Atraso30'] = df['Operacion'].map(atraso_dict['Atraso30'])
df['Atraso60'] = df['Operacion'].map(atraso_dict['Atraso60'])
df['Atraso90'] = df['Operacion'].map(atraso_dict['Atraso90'])
df['Atraso120'] = df['Operacion'].map(atraso_dict['Atraso120'])
df['Atraso150'] = df['Operacion'].map(atraso_dict['Atraso150'])
df['Atraso180'] = df['Operacion'].map(atraso_dict['Atraso180'])

# Calculate 'Mora Final % Anualizada' and 'Rentabilidad Anualizada'
def annualize_percentage(value, cant_cuotas):
    return value * (12 / cant_cuotas) if cant_cuotas else 0

df['Mora Final % Anualizada'] = df.apply(lambda row: annualize_percentage(row['Mora Final %'], row['Cant. Cuotas']), axis=1)

df['Rentabilidad'] = df.apply(lambda row: (row['Pagare'] - row['Capital actual'] - row['Mora Final']) / row['Capital actual'] if row['Capital actual'] != 0 else 0, axis=1)
df['Rentabilidad Anualizada'] = df.apply(lambda row: annualize_percentage(row['Rentabilidad'], row['Cant. Cuotas']), axis=1)

# Condense the DataFrame by keeping only the first row of each 'Operacion'
df_condensed = df.drop_duplicates(subset='Operacion', keep='first')

# Save the modified DataFrame to a new CSV file
output_file_path = '/Users/fabrizioferrari/Desktop/final boss/tlon/Training_Set_Columnas_Condensed.csv'
df_condensed.to_csv(output_file_path, index=False)

print(f"Data with 'Mora Final', 'Pagare', 'Mora Final %', 'Mora Final % Anualizada', 'Rentabilidad', 'Rentabilidad Anualizada', 'Atraso30', 'Atraso60', 'Atraso90', 'Atraso120', 'Atraso150', and 'Atraso180' saved to {output_file_path}")


/var/folders/6m/k7tj7j8j0z1_ry92lnh0blg00000gn/T/ipykernel_9060/674678322.py:84: DtypeWarning: Columns (17,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Data with 'Mora Final', 'Pagare', 'Mora Final %', 'Mora Final % Anualizada', 'Rentabilidad', 'Rentabilidad Anualizada', 'Atraso30', 'Atraso60', 'Atraso90', 'Atraso120', 'Atraso150', and 'Atraso180' saved to /Users/fabrizioferrari/Desktop/final boss/tlon/Training_Set_Columnas_Condensed.csv


In [4]:
#generar columnas y condensar:
#columnas test set condensado stripped

import pandas as pd

# Define the columns to retain
numerical_cols = ["Cant. Cuotas", "Capital actual", "Edad", "INGRESO_CLIENTE", "Valor Cuota", "MONTODESEMBOLSADO"]
categorical_cols = ["APORTA_IVA", "Aportaips", "Banca", "CALIFICACION_ANTERIOR", "CLIENTEFORMAL",
                    "EMPRESA_PUBLICA1_LAB", "EMPRESA_PUBLICA2_LAB", "HABILITA_PROD1_BNF", "HABILITA_PROD2_BNF",
                    "MARCA", "SEXO", "Tipo", "Departamento", "Sucursaltipo", "Medio", "Sucursal", "Canal"]

# Load the CSV file
file_path = '/Users/fabrizioferrari/Desktop/final boss/tlon/Test_Set.csv'
df = pd.read_csv(file_path)

# Combine numerical and categorical columns with 'Operacion' as the key column to retain
cols_to_keep = ['Operacion'] + numerical_cols + categorical_cols

# Strip the unnecessary columns
df_reduced = df[cols_to_keep]

# Condense the DataFrame by keeping only the first row of each 'Operacion'
df_condensed = df_reduced.drop_duplicates(subset='Operacion', keep='first')

# Save the modified DataFrame to a new CSV file
output_file_path = '/Users/fabrizioferrari/Desktop/final boss/tlon/Test_Set_Condensed_Stripped.csv'
df_condensed.to_csv(output_file_path, index=False)

print(f"Data with selected columns saved to {output_file_path}")


/var/folders/6m/k7tj7j8j0z1_ry92lnh0blg00000gn/T/ipykernel_9060/3316939386.py:14: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Data with selected columns saved to /Users/fabrizioferrari/Desktop/final boss/tlon/Test_Set_Condensed_Stripped.csv


In [5]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix, accuracy_score
import pickle

# Load the data
data = pd.read_csv('/Users/fabrizioferrari/Desktop/final boss/tlon/Training_Set_Columnas_Condensed.csv')

# Define the features and target
numerical_cols = ["Cant. Cuotas", "Capital actual", "Edad", "INGRESO_CLIENTE", "Valor Cuota", "MONTODESEMBOLSADO"]
categorical_cols = ["APORTA_IVA", "Aportaips", "Banca", "CALIFICACION_ANTERIOR", "CLIENTEFORMAL",
                    "EMPRESA_PUBLICA1_LAB", "EMPRESA_PUBLICA2_LAB", "HABILITA_PROD1_BNF", "HABILITA_PROD2_BNF",
                    "MARCA", "SEXO", "Tipo", "Departamento", "Sucursaltipo", "Medio", "Sucursal", "Canal"]
target_col = 'Mora Final %'

# Define the threshold for binarizing the target
threshold = 0.05  # Set this to the desired threshold

# Binarize the target based on the threshold
data[target_col] = (data[target_col] > threshold).astype(int)

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the model with CatBoostClassifier and hyperparameters
model = CatBoostClassifier(
    iterations=6000,     # Number of boosting iterations
    learning_rate=0.01, 
    l2_leaf_reg=3,# Learning rate
    depth=3,             # Depth of the tree
    verbose=100          # Verbosity of the training process
)

# Create and train the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', model)])

# Train the model
pipeline.fit(data.drop(columns=[target_col]), data[target_col])

# Save the model
with open('Tlön_Negofin.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

# Predictions for analysis
preds = pipeline.predict(data.drop(columns=[target_col]))
conf_matrix = confusion_matrix(data[target_col], preds)
accuracy = accuracy_score(data[target_col], preds)
total_rows = len(data)

# Confusion matrix analysis
tn, fp, fn, tp = conf_matrix.ravel()
conf_matrix_analysis = {
    "True Negative": tn,
    "False Positive": fp,
    "False Negative": fn,
    "True Positive": tp,
    "Total": total_rows,
    "Percent True Negative": tn / total_rows * 100,
    "Percent False Positive": fp / total_rows * 100,
    "Percent False Negative": fn / total_rows * 100,
    "Percent True Positive": tp / total_rows * 100,
    "Red Flag Percent": (fp + tp) / total_rows * 100,
    "Accuracy": accuracy * 100
}

# Output analysis
print("Confusion Matrix:\n", conf_matrix)
print("\nConfusion Matrix Analysis:\n", conf_matrix_analysis)


/var/folders/6m/k7tj7j8j0z1_ry92lnh0blg00000gn/T/ipykernel_9060/1521325947.py:11: DtypeWarning: Columns (17,60) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/Users/fabrizioferrari/Desktop/final boss/tlon/Training_Set_Columnas_Condensed.csv')


0:	learn: 0.6886377	total: 68.1ms	remaining: 6m 48s
100:	learn: 0.5023894	total: 743ms	remaining: 43.4s
200:	learn: 0.4755238	total: 1.38s	remaining: 39.9s
300:	learn: 0.4679680	total: 2.02s	remaining: 38.3s
400:	learn: 0.4640642	total: 2.67s	remaining: 37.3s
500:	learn: 0.4612628	total: 3.31s	remaining: 36.4s
600:	learn: 0.4589991	total: 4s	remaining: 35.9s
700:	learn: 0.4571693	total: 4.71s	remaining: 35.6s
800:	learn: 0.4555989	total: 5.35s	remaining: 34.8s
900:	learn: 0.4542168	total: 6s	remaining: 34s
1000:	learn: 0.4530603	total: 6.79s	remaining: 33.9s
1100:	learn: 0.4519653	total: 7.49s	remaining: 33.3s
1200:	learn: 0.4510071	total: 8.18s	remaining: 32.7s
1300:	learn: 0.4500439	total: 8.84s	remaining: 31.9s
1400:	learn: 0.4492409	total: 9.55s	remaining: 31.3s
1500:	learn: 0.4485155	total: 10.3s	remaining: 30.8s
1600:	learn: 0.4478370	total: 11s	remaining: 30.2s
1700:	learn: 0.4472414	total: 11.7s	remaining: 29.6s
1800:	learn: 0.4466776	total: 12.4s	remaining: 28.9s
1900:	learn: 

In [10]:
#code que hace load el pickle file, hace run el modelo con test set condensado stripped y hace output un csv de solo la operacion y score

import pandas as pd
import pickle
from sklearn.pipeline import Pipeline

# Load the pre-trained model
model_file_path = 'Tlön_Negofin.pkl'
with open(model_file_path, 'rb') as f:
    model = pickle.load(f)

# Load the test set data
test_data_file_path = '/Users/fabrizioferrari/Desktop/final boss/tlon/Test_Set_Condensed_Stripped.csv'
test_data = pd.read_csv(test_data_file_path)

# Assuming 'Operacion' is a unique identifier in the test set
operacion_col = test_data['Operacion']

# Retain 'Cant. Cuotas' for threshold calculation
cant_cuotas_col = test_data['Cant. Cuotas']

# Predict the probability of the positive class using the loaded model
probabilities = model.predict_proba(test_data)[:, 1]

## Define the dynamic threshold function lvl 1
#def calculate_threshold(cant_cuotas):
#    if cant_cuotas <= 12:
#        return 0.324
#    elif cant_cuotas <= 18:
#        return 0.53
#    elif cant_cuotas <= 24:
#        return 0.69
#    else:
#        return 0.69


## Define the dynamic threshold function lvl 2
#def calculate_threshold(cant_cuotas):
#    if cant_cuotas <= 12:
#        return 0.21
#    elif cant_cuotas <= 18:
#        return 0.385
#    elif cant_cuotas <= 24:
#        return 0.603
#    else:
#        return 0.603

# Define the dynamic threshold function lvl 3
def calculate_threshold(cant_cuotas):
    if cant_cuotas <= 12:
        return 0.16
    elif cant_cuotas <= 18:
        return 0.31
    elif cant_cuotas <= 24:
        return 0.55
    else:
        return 0.55
# Apply the dynamic threshold to determine the final predictions
thresholds = cant_cuotas_col.apply(calculate_threshold)
predictions = (probabilities >= thresholds).astype(int)

# Prepare the output DataFrame
output_df = pd.DataFrame({
    'Operacion': operacion_col,
    'Resultado': predictions
})

# Save the output to a new CSV file
output_file_path = '/Users/fabrizioferrari/Desktop/final boss/tlon/Resultados.csv'
output_df.to_csv(output_file_path, index=False)

print(f"Predictions saved to {output_file_path}")


Predictions saved to /Users/fabrizioferrari/Desktop/final boss/tlon/Resultados.csv


In [7]:
#########################HJHHJHHJHJHJH

import pandas as pd
import pickle
from sklearn.pipeline import Pipeline

# Load the pre-trained model
model_file_path = 'Tlön_Negofin.pkl'
with open(model_file_path, 'rb') as f:
    model = pickle.load(f)

# Load the test set data
test_data_file_path = '/Users/fabrizioferrari/Desktop/final boss/tlon/Test_Set_Condensed_Stripped.csv'
test_data = pd.read_csv(test_data_file_path)

# Assuming 'Operacion' is a unique identifier in the test set
operacion_col = test_data['Operacion']

# Predict the probability of the positive class using the loaded model
probabilities = model.predict_proba(test_data)[:, 1]

# Scale probabilities to a score from 1 to 1000
scores = 1000 - (probabilities * 1000).astype(int)

# Prepare the output DataFrame    ######################## cambiar ########ASGASGSDFGSDFGHSDFHS####################



#    rentabilidad bruta * (1-mora) = rentabilidad esperada anualizada ^ (cant cuo/12)
    
output_df = pd.DataFrame({
    'Operacion': operacion_col,
    'Resultado': scores
})

# Save the output to a new CSV file
output_file_path = '/Users/fabrizioferrari/Desktop/final boss/tlon/Resultados.csv'
output_df.to_csv(output_file_path, index=False)

print(f"Predictions saved to {output_file_path}")


Predictions saved to /Users/fabrizioferrari/Desktop/final boss/tlon/Resultados.csv


In [8]:
#hace stitch el csv de solo operacion y score con el test set complete sorted

import pandas as pd

# Load the first CSV file (source of columns to add)
file_path1 = '/Users/fabrizioferrari/Desktop/final boss/tlon/Resultados.csv'
df1 = pd.read_csv(file_path1)

# Load the second CSV file (destination CSV where columns will be added)
file_path2 = '/Users/fabrizioferrari/Desktop/final boss/tlon/Test_Set_Columnas.csv'
df2 = pd.read_csv(file_path2)

# List of columns to stitch from df1 to df2
columns_to_stitch = ['Resultado']  # Replace with actual column names

# Ensure 'Operacion' is present in both DataFrames
if 'Operacion' not in df1.columns or 'Operacion' not in df2.columns:
    raise ValueError("'Operacion' column must be present in both CSV files.")

# Merge df1 with df2 on the 'Operacion' column, keeping all rows from df2
df_merged = pd.merge(df2, df1[['Operacion'] + columns_to_stitch], on='Operacion', how='left')

# Save the merged DataFrame to a new CSV file
output_file_path = '/Users/fabrizioferrari/Desktop/final boss/tlon/Test_Set_Columnas_Resultados.csv'
df_merged.to_csv(output_file_path, index=False)

print(f"Columns from the first CSV stitched to the second CSV and saved to {output_file_path}")


/var/folders/6m/k7tj7j8j0z1_ry92lnh0blg00000gn/T/ipykernel_9060/2492625917.py:11: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(file_path2)


Columns from the first CSV stitched to the second CSV and saved to /Users/fabrizioferrari/Desktop/final boss/tlon/Test_Set_Columnas_Resultados.csv


In [9]:
import pandas as pd

# Load the data
file_path = '/Users/fabrizioferrari/Desktop/final boss/tlon/Test_Set_Columnas_Resultados.csv'
data = pd.read_csv(file_path)

#data = data[data['Cant. Cuotas'] <= 24 ]
#data = data[data['Cant. Cuotas'] > 18 ]


# Aggregate data by 'Operacion'
aggregated_data = data.groupby('Operacion').agg({
    'Mora Final': 'sum',
    'Pagare': 'sum',
    'Capital actual': 'sum',
    'Resultado': 'first',  # Assuming 'Resultado' is consistent across rows for the same 'Operacion'
    'Cant. Cuotas': 'first'  # Assuming 'Cant. Cuotas' is consistent across rows for the same 'Operacion'
}).reset_index()

# Function to calculate Mora Final %, Rentabilidad, and Rentabilidad Descontada
def calculate_metrics(df):
    mora_final_total = df['Mora Final'].sum()
    pagare_total = df['Pagare'].sum()
    capital_anterior_total = df['Capital actual'].sum()

    mora_final_percent = mora_final_total / pagare_total if pagare_total != 0 else 0
    rentabilidad = ((pagare_total - capital_anterior_total - mora_final_total) / capital_anterior_total) if capital_anterior_total != 0 else 0

    # Adjust Rentabilidad for loan duration to get Rentabilidad Descontada
    average_cant_cuotas = df['Cant. Cuotas'].mean()
    rentabilidad_descontada = rentabilidad * (12 / average_cant_cuotas) if average_cant_cuotas != 0 else 0

    return mora_final_percent, rentabilidad, rentabilidad_descontada

# Calculate metrics for the entire dataset
total_mora_final_percent, total_rentabilidad, total_rentabilidad_descontada = calculate_metrics(aggregated_data)

# Calculate metrics for 'Resultado' = 1
data_result_1 = aggregated_data[aggregated_data['Resultado'] == 1]
mora_final_percent_1, rentabilidad_1, rentabilidad_descontada_1 = calculate_metrics(data_result_1)

# Calculate metrics for 'Resultado' = 0
data_result_0 = aggregated_data[aggregated_data['Resultado'] == 0]
mora_final_percent_0, rentabilidad_0, rentabilidad_descontada_0 = calculate_metrics(data_result_0)

# Calculate the percentage of 'Operaciones' that got 1 in 'Resultado'
percent_result_1 = len(data_result_1) / len(aggregated_data) * 100

# Print the overall results
print("Overall Metrics:")
print(f"Mora Final %: {total_mora_final_percent:.2%}")
print(f"Rentabilidad: {total_rentabilidad:.2%}")
print(f"Rentabilidad Descontada: {total_rentabilidad_descontada:.2%}")
print(f"Percent of Operaciones with Resultado = 1: {percent_result_1:.2f}%")
print("\nMetrics for Resultado = 1:")
print(f"Mora Final %: {mora_final_percent_1:.2%}")
print(f"Rentabilidad: {rentabilidad_1:.2%}")
print(f"Rentabilidad Descontada: {rentabilidad_descontada_1:.2%}")
print("\nMetrics for Resultado = 0:")
print(f"Mora Final %: {mora_final_percent_0:.2%}")
print(f"Rentabilidad: {rentabilidad_0:.2%}")
print(f"Rentabilidad Descontada: {rentabilidad_descontada_0:.2%}")

# Calculate metrics for each unique value of 'Cant. Cuotas'
for cuotas in aggregated_data['Cant. Cuotas'].unique():
    data_cuotas = aggregated_data[aggregated_data['Cant. Cuotas'] == cuotas]
    mora_final_percent_cuotas, rentabilidad_cuotas, rentabilidad_descontada_cuotas = calculate_metrics(data_cuotas)
    print(f"\nMetrics for Cant. Cuotas = {cuotas}:")
    print(f"Mora Final %: {mora_final_percent_cuotas:.2%}")
    print(f"Rentabilidad: {rentabilidad_cuotas:.2%}")
    print(f"Rentabilidad Descontada: {rentabilidad_descontada_cuotas:.2%}")


/var/folders/6m/k7tj7j8j0z1_ry92lnh0blg00000gn/T/ipykernel_9060/4043120300.py:5: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


Overall Metrics:
Mora Final %: 13.00%
Rentabilidad: 28.00%
Rentabilidad Descontada: 25.19%
Percent of Operaciones with Resultado = 1: 0.00%

Metrics for Resultado = 1:
Mora Final %: 0.00%
Rentabilidad: 0.00%
Rentabilidad Descontada: nan%

Metrics for Resultado = 0:
Mora Final %: 0.00%
Rentabilidad: 0.00%
Rentabilidad Descontada: nan%

Metrics for Cant. Cuotas = 12:
Mora Final %: 9.01%
Rentabilidad: 27.56%
Rentabilidad Descontada: 27.56%

Metrics for Cant. Cuotas = 18:
Mora Final %: 15.60%
Rentabilidad: 31.09%
Rentabilidad Descontada: 20.72%

Metrics for Cant. Cuotas = 10:
Mora Final %: 8.15%
Rentabilidad: 12.41%
Rentabilidad Descontada: 14.90%

Metrics for Cant. Cuotas = 24:
Mora Final %: 19.38%
Rentabilidad: 47.26%
Rentabilidad Descontada: 23.63%

Metrics for Cant. Cuotas = 6:
Mora Final %: 6.23%
Rentabilidad: 6.36%
Rentabilidad Descontada: 12.73%

Metrics for Cant. Cuotas = 8:
Mora Final %: 8.28%
Rentabilidad: 8.33%
Rentabilidad Descontada: 12.49%

Metrics for Cant. Cuotas = 15:
Mora